In [ ]:
%pip install -q --no-warn-conflicts malariagen_data
%pip install -q --no-warn-conflicts petl
%pip install -U kaleido
%pip install -U plotly
import kaleido
kaleido.get_chrome_sync()

In [2]:
import allel
import malariagen_data
import numpy as np
import pandas as pd
import dask
import dask.array as da
# silence some dask warnings
dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.diagnostics.progress import ProgressBar

In [3]:
# plotting setup
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.legend import Legend

import kaleido
import plotly.express as px
import plotly.io as pio
import PIL
import io
from IPython.display import Image
import plotly.graph_objects as go

In [4]:
import random
import functools
import petl as ptl
import itertools
import scipy
from collections import Counter

In [5]:
#Mounting Google Drive
import os
from google.colab import drive
drive.mount("drive")

# make dir
results_dir = "drive/MyDrive"
os.makedirs(results_dir, exist_ok=True)

Mounted at drive


In [6]:
## Importing malariagen data set
ag3 = malariagen_data.Ag3("gs://vo_agam_release_master_us_central1/", pre=True)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release_master_us_central1/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12, 3.13, 3.14, 3.15
Results cache           : None
Cohorts analysis        : 20250502
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 15.3.0
Client location         : Utah, United States (Google Cloud us-west3)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v15.3.0/Ag3.html

In [7]:
df_samples=ag3.sample_metadata(sample_sets='3.11', sample_query='country=="Burkina Faso"')
sets = list(df_samples.sample_set.unique())
transcript_ace1='AGAP001356-RA'

## Computting diversity stats

In [8]:
#To access to the genotypes within the 3L chromosomes
ds_snps = ag3.snp_calls(region="2R:3484107-3495790", sample_sets=sets)

# Take some SNP in the X chromosomes
ds_pos = allel.SortedIndex(ds_snps['variant_position'].values)

# To filter the SNP dataset and warp the dataset to GT array
filt = 'gamb_colu_arab'
filt_val = ds_snps[f"variant_filter_pass_{filt}"].values

## Compute genotype & count the number of alleles
with ProgressBar():
  ds_gt_filtered = allel.GenotypeDaskArray(ds_snps["call_genotype"][filt_val].data).compute()
  ds_ac = ds_gt_filtered.count_alleles()

is_var = ds_ac.is_variant()
pos_filt = allel.SortedIndex(ds_snps['variant_position'][filt_val].values)
pos_filt_var = pos_filt[is_var]

[########################################] | 100% Completed | 3.35 s


In [9]:
## segregating sites
print(ds_ac.count_segregating(), 'segregating sites')
print(ds_ac.count_variant(), 'variants sites')
print(ds_ac.is_biallelic().sum(), 'biallelic sites')
print(ds_ac.count_segregating()-ds_ac.is_biallelic().sum(), 'sites are not biallelic')
print(100*(ds_ac.count_segregating()-ds_ac.is_biallelic().sum())/ds_ac.count_segregating(), '% sites are not biallelic')

4179 segregating sites
4180 variants sites
3367 biallelic sites
812 sites are not biallelic
19.430485762144052 % sites are not biallelic


In [10]:
## nucleotide diversity vs tajima's D and WT
pi = allel.sequence_diversity(pos_filt, ds_ac)
D = allel.tajima_d(ds_ac, pos_filt)
w_theta = allel.watterson_theta(pos_filt, ds_ac)
print('pi = ', pi)
print('D = ', D)
print('w_theta = ', w_theta)

pi =  0.008109093894413803
D =  -2.397864163736441
w_theta =  0.0460333352403403


# Frequencies

In [11]:
## define samples cohorts
cohorts, cohorts_, pop_list, pop_list1 = {}, {}, [],[]
for loca in df_samples.location.unique():
    loca_sample = df_samples.query(f"location == '{loca}'")
    for species in loca_sample.aim_species.unique():
      sp_sample=loca_sample.query(f'aim_species=="{species}"')
      key2 = loca[:4]+'_'+species[:3]
      key3 = loca+'[An. '+species+']'
      cohorts_[key2] = f"country == 'Burkina Faso' and location=='{loca}' and aim_species == '{species}'"
      pop_list1.append(key3)
      for year in sp_sample.year.unique():
        key = loca[:4]+'_'+species[:3]+'_'+str(year)
        key1 = loca+'[An. '+species+' ('+ str(year)+')]'
        cohorts[key] = f"country == 'Burkina Faso' and location=='{loca}' and aim_species == '{species}' and year == {year}"
        pop_list1.append(key1)

## Cohorts size
cohorts_size, cohorts_size_ = {}, {}
for coh in cohorts.keys():
  cohorts_size[coh] = df_samples.query(f'{cohorts[coh]}').shape[0]
for coh in cohorts_.keys():
  cohorts_size_[coh] = df_samples.query(f'{cohorts_[coh]}').shape[0]

In [42]:
## Frequencies
ds_freq_ace1 = ag3.aa_allele_frequencies(transcript=transcript_ace1, cohorts=cohorts, sample_sets='3.11',
                                         sample_query="country == 'Burkina Faso'",min_cohort_size=5)
ds_freq_ace1.reset_index(inplace=True)

Load SNP genotypes:   0%|          | 0/70 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/23 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/7797 [00:00<?, ?it/s]

In [82]:
ds_freq_ace1.to_csv(f'{results_dir}/sanger_analyses/resistance/ACE1/data/ace1_freq.csv')

In [14]:
#ds_freq_ace1['max_af'].describe()

In [15]:
def ds_freq_tab(ds):
  #extract cohorts into a dataframe
  cohort_vars = [v for v in ds if v.startswith("cohort_")]
  df_cohorts = ds[cohort_vars].to_dataframe()
  df_cohorts.columns = [c.split("cohort_")[1] for c in df_cohorts.columns]

  variant_labels = ds["variant_label"].values
  dfs = []
  for cohort_index, cohort in enumerate(df_cohorts.itertuples()):
    ds_cohort = ds.isel(cohorts=cohort_index)
    dict_df =  {"taxon": cohort.taxon, "area": cohort.area, "date": cohort.period_start, "period": str(cohort.period),
                "sample_size": cohort.size,"variant": variant_labels, "count": ds_cohort["event_count"].values,"nobs": ds_cohort["event_nobs"].values,
                "frequency": ds_cohort["event_frequency"].values, "frequency_ci_low": ds_cohort["event_frequency_ci_low"].values,
                "frequency_ci_upp": ds_cohort["event_frequency_ci_upp"].values
                }
    df = pd.DataFrame(dict_df)
    dfs.append(df)

  df_events = pd.concat(dfs, axis=0).reset_index(drop=True)
  df_events = df_events.query("nobs > 0")

  # Frequencies stats
  frq = df_events["frequency"]
  frq_ci_low = df_events["frequency_ci_low"]
  frq_ci_upp = df_events["frequency_ci_upp"]
  df_events["frequency_error"] = frq_ci_upp - frq
  df_events["frequency_error_minus"] = frq - frq_ci_low

  return df_events

In [46]:
ds_aafreq_ace1 = ag3.aa_allele_frequencies_advanced(transcript=transcript_ace1, area_by="location", period_by="year",
                                                    sample_sets='3.11', sample_query="country == 'Burkina Faso'",
                                                    min_cohort_size=5)
df_events_ace1 = ds_freq_tab(ds_aafreq_ace1)
df_events_ace1['populations']=[str(x)[:4]+'_'+str(y)[:4]+'_'+str(z) for x,y,z in zip(df_events_ace1.area,df_events_ace1.taxon,df_events_ace1.period)]

Load SNP genotypes:   0%|          | 0/70 [00:00<?, ?it/s]

Compute SNP allele frequencies:   0%|          | 0/23 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/7797 [00:00<?, ?it/s]

In [47]:
df_vars = df_events_ace1.query("variant == 'G280S (2R:3,492,074 G>A)' and period =='2022'").reset_index(drop=True)
#df_vars

In [48]:
fig1 = px.bar(data_frame=df_vars,height=450, width=600, template='plotly_white',
              x='area',y="frequency", error_y="frequency_error",
              error_y_minus="frequency_error_minus", color='taxon', barmode='group')
fig1.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig1.update_yaxes(range=[0, 1.0], ticks="outside", col=1)

In [49]:
img_bytes=fig1.to_image(format="png", width=800, height=350, scale=2)
open_img = PIL.Image.open(io.BytesIO(img_bytes))
open_img.save('drive/MyDrive/sanger_analyses/resistance/ACE1/savefig/ace1_G280S_pop.png', format='png',dpi=(300,300))

# ACE1 gene CNV

In [50]:
# define function to find metabolic resiatnce gene cluster in each contig
def get_cnv_gene(region, sample_sets, sample_query, gene=None, gene_id=None):
  cnv_ = ag3.gene_cnv(region = region, sample_sets=sample_sets, sample_query=sample_query)

  ## col names
  columns = ['gene_start', 'gene_end','gene_name', 'gene_description']

  # to def
  df_cnv = cnv_['gene_contig'].to_dataframe()

  # insert other col name
  for col in columns:
    df = cnv_[f'{col}']
    df_cnv.insert(columns.index(f'{col}')+1, f'{col}', df)

  #find gene information
  gcontig, gname, gid, gstart, gend, gene_desc = [],[],[],[], [], []
  for contig,name, id, start, end, desc in zip(df_cnv.gene_contig,df_cnv.gene_name, df_cnv.gene_id, df_cnv.gene_start, df_cnv.gene_end,df_cnv.gene_description):
    if str(name)[:3]==f'{gene}':
      gname.append(name), gid.append(id), gstart.append(start), gend.append(end), gcontig.append(contig), gene_desc.append(desc)
    elif str(id)==f'{gene_id}':
        gname.append(name), gid.append(id), gstart.append(start), gend.append(end), gcontig.append(contig), gene_desc.append(desc)

  #create dataFrame
  df_gene = pd.DataFrame(zip(gcontig,gstart, gend, gname, gid, gene_desc), columns=['contig','start','stop', 'name', 'id', 'gene_description'])

  return df_gene

In [52]:
## computes frequencies
region, df_list = ['X', '2R', '2L', '3R', '3L'],[]
## Ace1
for item in region:
  df_gene = get_cnv_gene(region=f'{item}', sample_sets=sets, sample_query='country=="Burkina Faso"', gene='ACE')
  if df_gene.empty == False:
    df_list.append(df_gene)

df_ace = pd.concat(df_list)

### gene list
ace_id_list = list(df_ace.id)

## compute cnv frequencies in each gene cluster
ace_cnv_freq = ag3.gene_cnv_frequencies(
    region=ace_id_list,
    cohorts=cohorts,
    sample_sets=sets,
    sample_query='country=="Burkina Faso"',
    min_cohort_size=5
    )

## compute time serie cnv frequencies
ace_cnv_freq_ts = ag3.gene_cnv_frequencies_advanced(
    region=ace_id_list,
    area_by="location",
    period_by="year",
    sample_sets=sets,
    sample_query="country == 'Burkina Faso'",
    min_cohort_size=5
    )
df_events_ace = ds_freq_tab(ace_cnv_freq_ts)
df_events_ace['populations']=[str(x)[:4]+'_'+str(y)[:4]+'_'+str(z) for x,y,z in zip(df_events_ace.area,df_events_ace.taxon,df_events_ace.period)]
## def columns
freq_columns = [col for col in ace_cnv_freq.columns if col.startswith('frq_')]

## remove unamed gene data
ace_cnv_freq = ace_cnv_freq.reset_index().query('gene_name.notna()', engine='python')


Load CNV HMM data:   0%|          | 0/133 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/1063 [00:00<?, ?it/s]

Load CNV HMM data:   0%|          | 0/289 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/3668 [00:00<?, ?it/s]

Load CNV HMM data:   0%|          | 0/211 [00:00<?, ?it/s]

...

Compute modal gene copy number:   0%|          | 0/1 [00:00<?, ?it/s]

Load CNV HMM data:   0%|          | 0/55 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/1 [00:00<?, ?it/s]

Load CNV HMM data:   0%|          | 0/55 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
#df_events_ace

In [23]:
#ace_cnv_freq

In [53]:
fig2 = px.bar(data_frame=df_events_ace,height=450, width=600, template='plotly_white',
              x='area',y="frequency", error_y="frequency_error",
              error_y_minus="frequency_error_minus", color='taxon', barmode='group')
fig2.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig2.update_yaxes(range=[0, 1.0], ticks="outside", col=1)

## SNP+CNV

In [54]:
df_ace1=pd.concat([df_events_ace, df_vars])
df_ace1['location_1'] = [loc[:4]+f' (n={n})' for loc, n in zip(df_ace1.area, df_ace1.sample_size)]
df_ace1['location'] = [loc[:4] for loc in df_ace1.area]

In [57]:
fig3 = px.bar(data_frame=df_ace1.query('taxon=="gambiae"'),height=450, width=600, template='plotly_white',
              x='area',y="frequency", error_y="frequency_error",
              error_y_minus="frequency_error_minus", color='variant', barmode='group')
fig3.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig3.update_yaxes(range=[0, 1.0], ticks="outside", col=1)

In [58]:
fig4 = px.bar(data_frame=df_ace1,height=450, width=1000, template='plotly_white',
              x='area',y="frequency", error_y="frequency_error",facet_col='taxon',
              error_y_minus="frequency_error_minus", color='variant', barmode='group')
fig4.update_traces(error_y=dict(thickness=1, width=4, color="purple"))
fig4.update_layout(title="ACE1 resistance",
                   xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray',  ticks="outside"),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   xaxis2=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis2=dict(showgrid=False),
                   xaxis3=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis3=dict(showgrid=False))
fig4.update_yaxes(range=[0, 1.0])

In [59]:
fig5 = px.bar(data_frame=df_ace1.query('frequency>0'),height=450, width=800, template='plotly_white',
              x='location',y="frequency", error_y="frequency_error",facet_col='taxon',
              error_y_minus="frequency_error_minus", color='variant', barmode='group')
fig5.update_traces(error_y=dict(thickness=1, width=4, color="purple"))
fig5.update_layout(title="ACE1 CNV/SNP frequencies",
                   xaxis=dict(showgrid=False,title=' ', showline=True, linewidth=1, linecolor='gray',  ticks="outside"),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   xaxis2=dict(showgrid=False,title=' ', showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis2=dict(showgrid=False))
fig5.update_yaxes(range=[0, 1.0])

In [29]:
img_bytes=fig5.to_image(format="png", width=800, height=350, scale=2)
open_img = PIL.Image.open(io.BytesIO(img_bytes))
open_img.save('drive/MyDrive/sanger_analyses/resistance/ACE1/savefig/ace1_G280S_amp.png', format='png',dpi=(300,300))

In [68]:
fig71 = px.bar(data_frame=df_ace1.query('frequency>0'),height=380, width=900, template='plotly_white',
              x='location',y="frequency", error_y="frequency_error",facet_col='taxon',
              error_y_minus="frequency_error_minus", color='variant', barmode='group',
              labels={"location": ' ', "frequency": 'Allele frequencies', "variant":'ace1R variants'})
fig71.update_traces(error_y=dict(thickness=1, width=4, color="purple"))
fig71.update_layout(title=" ",
                   xaxis=dict(showgrid=False,title='Sampling sites', tickangle= -35, showline=True, linewidth=1, linecolor='gray',  ticks="outside"),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   xaxis2=dict(showgrid=False,title='Sampling sites', tickangle= -35, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis2=dict(showgrid=False,showline=True,linecolor='gray', ticks="outside"))
names = {'taxon=coluzzii':'Acol','taxon=gambiae':'Agam'}
fig71.for_each_annotation(lambda a: a.update(text=names[a.text]))
#fig71.add_annotation(x=0.5,y=-0.28,text="Sampling sites",showarrow=False,xref="paper",yref="paper",font=dict(size=14))
fig71.update_yaxes(range=[0, 1.0])

In [73]:
fig7 = px.bar(data_frame=df12,height=380, width=900, template='plotly_white',
              x='location',y="frequency", error_y="error_freq",facet_col='taxon',
              error_y_minus="frequency_error_minus", color='variant', barmode='group',
              labels={"location": ' ', "frequency": 'Allele frequencies', "variant":'ace1R variants'})
fig7.update_traces(error_y=dict(thickness=1, width=4, color="purple"))
fig7.update_layout(title=" ",
                   xaxis=dict(showgrid=False,title='Sampling sites', tickangle= -35, showline=True, linewidth=1, linecolor='gray',  ticks="outside"),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   xaxis2=dict(showgrid=False,title='Sampling sites', tickangle= -35, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis2=dict(showgrid=False,showline=True,linecolor='gray', ticks="outside"))
names = {'taxon=coluzzii':'Acol','taxon=gambiae':'Agam'}
fig7.for_each_annotation(lambda a: a.update(text=names[a.text]))
#fig7.add_annotation(x=0.5,y=-0.28,text="Sampling sites",showarrow=False,xref="paper",yref="paper",font=dict(size=14))
fig7.update_yaxes(range=[0, 1.0])

In [80]:
df12 = df_ace1.query('date==2022')
df12['frequency_error'] = np.sqrt(df12['frequency']*(1-df12['frequency'])/df12['sample_size'])
df12.to_csv(f'{results_dir}/sanger_analyses/resistance/ACE1/data/ace1_cnv_snp_2022.csv')

In [79]:
np.sum(df12['sample_size'])

np.int64(1092)

In [81]:
img_bytes=fig7.to_image(format="png", width=800, height=350, scale=2)
open_img = PIL.Image.open(io.BytesIO(img_bytes))
open_img.save('drive/MyDrive/sanger_analyses/resistance/ACE1/savefig/ace1_G280S_amp1.png', format='png',dpi=(300,300))

In [32]:
#df_ace1.query('frequency>0')

In [33]:
fig6 = px.bar(data_frame=df_ace1.query('taxon!="arabiensis"'),height=450, width=900, template='plotly_white',
              x='location',y="frequency", error_y="frequency_error",facet_col='taxon',
              error_y_minus="frequency_error_minus", color='variant', barmode='group')
fig6.update_traces(error_y=dict(thickness=1, width=4, color="purple"))
fig6.update_layout(title="ACE1 CNV/SNP frequencies",
                   xaxis=dict(showgrid=False,title=' ', showline=True, linewidth=1, linecolor='gray',  ticks="outside"),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   xaxis2=dict(showgrid=False,title=' ', showline=True, linewidth=1, linecolor='gray', ticks="outside"),
                   yaxis2=dict(showgrid=False))
fig6.update_yaxes(range=[0, 1.0])

In [34]:
img_bytes=fig6.to_image(format="png", width=800, height=350, scale=2)
open_img = PIL.Image.open(io.BytesIO(img_bytes))
open_img.save('drive/MyDrive/sanger_analyses/resistance/ACE1/savefig/ace1_G280S_amp_lt.png', format='png',dpi=(300,300))

In [35]:
df_ace1.to_csv(f'{results_dir}/sanger_analyses/resistance/ACE1/data/ace1_cnv_snp.csv')

## data warping

In [36]:
df1_ace1=df_ace1.copy()
df_samples=ag3.sample_metadata(sample_sets='3.11', sample_query='country=="Burkina Faso"')
df_items = df_samples.groupby(['location','latitude','longitude']).size()
lon_lat = {}
for key in df_items.to_dict().keys():
  lon_lat[key[0]]=key[1:]
latitude, longitude = [],[]
df2_ace1=df1_ace1.sort_values(by='area')
for zone in df2_ace1.area.unique():
  if zone in lon_lat.keys():
    print(True, zone,lon_lat[zone][0],lon_lat[zone][1])
  else:
    print(False)
  for z in [itex for itex in range(df2_ace1.query(f'area=="{zone}"').shape[0])]:
    latitude.append(lon_lat[zone][0])
    longitude.append(lon_lat[zone][1])

print(len(latitude),len(longitude), df2_ace1.shape)
df2_ace1['LAT']=latitude
df2_ace1['LON']=longitude

df2_ace1['aa_change'] = [var.split(' ')[0] if var.startswith('G280S') else 'ACE1_amp' for var in df2_ace1.variant]

True Bana Village 11.234 -4.473
True Gama 12.003 1.762
True Nagare 12.927 -0.142
True Nassan 13.028 -3.014
True Ouro-Hesso 14.375 -0.128
True Po-Dongo 11.219 -1.02
True Sideradougou 10.678 -4.256
True Souroukoudinga 11.236 -4.537
30 30 (30, 16)


In [37]:
df3_ace1 = pd.pivot_table(df2_ace1, values=['count'], columns=['aa_change'], index=['populations','area','taxon','LAT','LON','nobs']).droplevel(0, axis=1).reset_index()
#df4_ace1 = pd.pivot_table(df3_ace1, values=['count'], columns=['aa_change'], index=['area','lat','lon','nobs']).droplevel(0, axis=1).reset_index()
df3_ace1.set_index('populations', inplace=True)
df3_ace1.to_excel(f'{results_dir}/sanger_analyses/resistance/ACE1/data/ace1_cnv_snp_warped.xlsx')

## Summary of the stats

In [38]:
df_ace1.query('taxon=="coluzzii"').groupby(['variant'])['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
variant,,,,,,,,
AGAP001356 (ACE1) amp,7.0,0.020929,0.029978,0.0,0.0,0.0,0.034790,0.076923
"G280S (2R:3,492,074 G>A)",8.0,0.006777,0.009933,0.0,0.0,0.0,0.013714,0.025000


In [39]:
df_ace1.query('taxon=="gambiae"').groupby(['variant'])['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
variant,,,,,,,,
AGAP001356 (ACE1) amp,4.0,0.226569,0.070915,0.125000,0.204327,0.251748,0.27399,0.277778
"G280S (2R:3,492,074 G>A)",5.0,0.131535,0.038579,0.074074,0.115385,0.138889,0.15625,0.173077


In [40]:
df_samples.groupby(['taxon', 'location', 'year']).size()

taxon       location        year
arabiensis  Gama            2022     13
            Nagare          2022      9
            Nassan          2022     13
            Ouro-Hesso      2022     10
            Po-Dongo        2022     11
            Sideradougou    2022      8
            Souroukoudinga  2021      1
                            2022      7
coluzzii    Bana Village    2020      2
                            2021     23
                            2022    120
            Degue-Degue     2022      4
            Gama            2022     39
            Nagare          2022    100
            Nassan          2022     32
            Ouro-Hesso      2022     61
            Po-Dongo        2022     20
            Sideradougou    2022     10
            Souroukoudinga  2021      5
                            2022     61
gambiae     Bana Village    2020      1
                            2022      4
            Degue-Degue     2022      5
            Gama            2022     27
            Nagare          2022     13
            Nassan          2022      2
            Po-Dongo        2022     18
            Sideradougou    2022     26
            Souroukoudinga  2021      2
                            2022     16
unassigned  Degue-Degue     2022      1
            Ouro-Hesso      2022      1
dtype: int64